In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

## preparing the data using PyTorch
    1. extract-get the data from Fashion MNIST IMAGE DATA 
    2. Transform put the data in a tensor 
    3. Load- put the data into an object to make it accessible


In [ ]:
# this piece of code is unnecessary if we're using a  standard dataset already available in torchvision.datasets 
# but its a great example to show whats happening under the hood in pytorch
# to create a dataset in pytorch we need to extend the torch.utils.data.Dataset class
# and implement the following methods:

class OHLC(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
    def __getitem__(self,index):
        r = self.data.iloc[index]
        label = torch.tensor(r.is_up_day, dtype=torch.long)
        sample = self.normalize(torch.tensor([r.open, r.high, r.low, r.close]))
        return sample,label
    def __len__(self):
        return len(self.data)
# we can create a dataset object by passing the csv file as an argument to the constructor
# and then we can use the dataset object to create a dataloader object by passing it to the DataLoader class
# and then we can use the dataloader object to iterate over the dataset in batches.
# we can also use the dataloader object to get the number of samples in the dataset.
# we can also use the dataloader object to get the number of batches in the dataset
